# Week 11 - Supervised Learning 

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

This week we will be applying some common supervised learning methods to classify the Iris dataset.
    
I have provided links to online tutorials for you learn how to use some basic scikit-learn classifiers by yourself. 
    
In the tutorial we will discuss how to assess the quality of models that you generate using the confusion matix and a number of metrics discussed in lectures.

</div>

## Classification Algorithms 

[Intro to Machine Learning](https://scikit-learn.org/stable/tutorial/basic/tutorial.html)

In general, a learning problem considers a set of n samples of data and then tries to predict properties of unknown data. If each sample is more than a single number and, for instance, a multi-dimensional entry (aka multivariate data), it is said to have several attributes or features.

Learning problems fall into a few categories:

- **Supervised learning**, in which the data comes with additional attributes that we want to predict [Click here to go to the scikit-learn supervised learning page](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning).This problem can be either:

 - **Classification**: samples belong to two or more classes and we want to learn from already labeled data how to predict the class of unlabeled data. An example of a classification problem would be handwritten digit recognition, in which the aim is to assign each input vector to one of a finite number of discrete categories. Another way to think of classification is as a discrete (as opposed to continuous) form of supervised learning where one has a limited number of categories and for each of the n samples provided, one is to try to label them with the correct category or class.
 - **Regression**: if the desired output consists of one or more continuous variables, then the task is called regression. An example of a regression problem would be the prediction of the length of a salmon as a function of its age and weight.

- **Unsupervised learning**, in which the training data consists of a set of input vectors x without any corresponding target values. The goal in such problems may be to discover groups of similar examples within the data, where it is called clustering, or to determine the distribution of data within the input space, known as density estimation, or to project the data from a high-dimensional space down to two or three dimensions for the purpose of visualization [Click here to go to the Scikit-Learn unsupervised learning page](https://scikit-learn.org/stable/unsupervised_learning.html#unsupervised-learning).



## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import neighbors
from matplotlib.colors import ListedColormap
import pandas as pd
import numpy as np
import os
import requests
from IPython.core.display import HTML

In [ ]:
files = ['CM.png','CM2.png','errorTypes.png','irises.jpeg']

for filename in files:
    url = f'https://github.com/melbournebioinformatics/COMP90014/blob/main/data/2023/Workshop_11/media/{filename}?raw=true'
    fetch_file(url,outpath='src/media')

# Data

## Import the dataset
To learn about classifiers will will be making use of the **sklearn library**. This library has some awesome tutorials and example datasets. One such dataset is the **Iris dataset**. The Iris flower data set or Fisher's Iris data set is a multivariate data set introduced by the British statistician and biologist Ronald Fisher in his 1936 paper *The use of multiple measurements in taxonomic problems as an example of linear discriminant analysis*. 

The data set consists of 50 samples from each of **three species** of Iris (*Iris setosa, Iris virginica and Iris versicolor*). 

**Four features** were measured from each sample: the length and the width of the sepals and petals, in centimeters.

<img src = "src/media/irises.jpeg"  width= "500"/>

In [ ]:
# Load in the dataset
from sklearn import datasets
iris = datasets.load_iris()
#print(iris.DESCR)

## Explore the data

In [ ]:
# Let's unpack the data and associated labels
petal_data = iris.data
labels = iris.target
target_names = iris.target_names
print('target names:\n', iris.target_names)
print('\nFirst 10 rows of data:\n', petal_data[:10])
print('\ntarget labels:', labels)

## Working with Numpy arrays

Each record in our dataset comprises measurements of petal length and width, and sepel length and width. These values are stored in a numpy array.

Arrays are an efficient way to work with large multidimentional datasets. Let's explore how we can interact with them.

For more info on working with arrays and all the cool things they can do check out the Numpy docs or the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html).

In [ ]:
# Our array has 2 dimensions
petal_data.ndim

In [ ]:
# The attribute 'shape' tells us the size of the 2 dimensions of our array 
# There are 150 lists each with 4 items.
petal_data.shape

In [ ]:
# There a 600 values in our array (150 * 4)
petal_data.size

#### Array indexing - access single elements

In [ ]:
# Array indexing - access single elements
# Access the first record in our dataset
petal_data[0]

In [ ]:
# Lower dims in our array can be accessed by chaining index calls
print( petal_data[0][0] )

# Or by comma separated indexing of each dim
print( petal_data[0,0] )

#### Slicing multidimentional arrays

In [ ]:
# Slicing works the same as for lists in python using [start:stop:step] notation
# We can slice each dimension of the array simultaneously using commas to separate the slices

# The first 5 records
petal_data[:5]

In [ ]:
# The first 2 values from each of the first 5 records as a sub-array
petal_data[:5,:2]

#### Accessing single rows or columns

In [ ]:
# Combining slices and indexing 
# return first value from each of the first 5 records
petal_data[:5,0]

## Visualising the data

In [ ]:
# We can perform a PCA to get a look at the data 
pca = PCA(n_components=2)
X_pca = pca.fit_transform(petal_data)

print(f'explained variance ratio (first two components): {str(pca.explained_variance_ratio_)}')

In [ ]:
# Plotting the PCA
plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_pca[labels == i, 0], X_pca[labels == i, 1], color=color, alpha=.8, lw=lw, label=target_name)
    
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of IRIS dataset')

### Training set and testing set

Machine learning is about learning some properties of a data set and then testing those properties against another data set. A common practice in machine learning is to evaluate an algorithm by splitting a data set into two. We call one of those sets the **training set**, on which we learn some properties; we call the other set the **testing set**, on which we test the learned properties.


### Split training and test data
To properly evaluate our model, we need to split the data into training and test sets.
The 'petal_data' is our data, and the 'labels' are each datapoint's class. 
Remember we want to guess (classify) a new iris flower to its correct label (iris flower type) given its petal data.

train_test_split will need to be given the petal_data, and the labels so it can split the data and labels in the same manner.
look up the [train_test_split documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to find how to do this.

In [ ]:
from sklearn.model_selection import train_test_split

# Create training petal_data, training labels, test petal_data, test lables using train_test_split()

data_train, data_test, label_train, label_test = train_test_split(petal_data, labels, test_size=0.6, random_state=10)

print(data_train[:10], '\n')
print(data_test[:10], '\n')

print(label_train[:10], '\n')
print(label_test[:10], '\n')

# Classifying the data

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

### Naive Bayes

Follow this link to learn about the scikit learn implementation of [Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html).
    
<b>Challange:</b> Copy the code from the sklearn tutorial and see if you can perform Gaussian Naive Bayes classification on the Iris dataset
    
</div>



In [ ]:
# data_train, data_test, label_train, label_test

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

# Assign the predicted labels to the variable pred_NB
# pred_NB =
### BEGIN SOLUTION
pred_NB = gnb.fit(data_train, label_train).predict(data_test)

print('true:', label_test)
print('pred:', pred_NB )

print("Number of mislabeled points out of a total %d points : %d" % (data_test.shape[0], (label_test != pred_NB).sum()))
### END SOLUTION


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">


### K Nearest Neighbours

Sample usage of [Nearest Neighbors classification](https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py). It will plot the decision boundaries for each class.
    
<b>Challange:</b> Copy the code from the tutorial and see if you can perform Nearest Neighbours classification on the Iris dataset.
    
</div>

In [ ]:
# data_train, data_test, label_train, label_test

from sklearn.neighbors import KNeighborsClassifier

# Assign the predicted labels to the variable pred_KNN
# pred_KNN =

### BEGIN SOLUTION
n = 15

clf = KNeighborsClassifier(n_neighbors=n, weights='uniform')

clf.fit(data_train, label_train)

pred_KNN = clf.predict(data_test)

print('true:', label_test)
print('pred:', pred_KNN)

print("Number of mislabeled points out of a total {} points : {}".format(data_test.shape[0], (label_test != pred_KNN).sum()))
### END SOLUTION 




<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

### Support Vector Machine

Comparison of different [linear SVM classifiers](https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html) on a 2D projection of the iris dataset. 
    
We will only consider the first 2 features of this dataset:

- Sepal length
- Sepal width
    
<b>Challange:</b> Copy and paste the code from this tutorial and see if you can perform SVM classification on the Iris dataset.

Note: See exercise in the link above for instructions on how to compare different flavours of SVM.
    
</div>


In [ ]:
from sklearn import svm


# Assign the predicted labels to the variable pred_SVM
# pred_SVM =

### BEGIN SOLUTION
clf = svm.SVC()
clf.fit(data_train[:,[0,1]], label_train)
pred_SVM = clf.predict(data_test[:,[0,1]])

# get support vectors
clf.support_vectors_

# get indices of support vectors
clf.support_

# get number of support vectors for each class
clf.n_support_

print('true:', label_test)
print('pred:', pred_SVM)

print("Number of mislabeled points out of a total {} points : {}".format(data_test.shape[0], (label_test != pred_SVM).sum()))
### END SOLUTION

# Assessing Models

## Graph Confusion Matrix Generation 

By definition a confusion matrix $C$ is such that $C_ij$ is equal to the number of observations known to be in group $i$ but predicted to be in group $j$.

Which is fancy jargon for this guy: 

<img src = "src/media/CM.png"  width= "500"/>

Which we can use to derive information about how well our classifiers performed

<img src = "src/media/errorTypes.png" width= "400"/>

For example, lets say we have used one of the classifiers listed above on our training data and now we will asses how well it did by testing it on a separate dataset with 10 samples which results in the following confusion matrix: 

<img src = "src/media/CM2.png" width= "400"/>


### Making the confusion matrix with Python

In python, you can code this with the following: 

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = ["T", "T", "T", "T", "F", "F", "F", "F", "F", "F"]
y_pred = ["T", "T", "T", "F", "T", "T", "F", "F", "F", "F"]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel() 
# Note: ravel just combines y_true and y_pred. Needed for confusion_matrix function.
print(f'tn: {tn}\nfp: {fp}\nfn: {fn}\ntp: {tp}')

### Confusion matrix for multiple classes
Okay cool. But for our example, there are 3 classes. How do we calculate these statistics?<br>

We calculate each statistic for a single class. <br>

For example, we would find TP, TN, FP, FP for iris setova. Then we would do the same for iris virginica etc.

In [ ]:
import seaborn as sns

y_true = [1, 0, 0, 0, 2, 2, 1, 0, 1, 0, 2, 0, 1, 2, 0, 0, 1, 0, 2, 0, 1]
y_pred = [0, 0, 0, 1, 2, 1, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 1, 1, 1, 0, 1]

cmatrix = confusion_matrix(y_true, y_pred)
labels = ['setosa', 'versicolor', 'virginica']
df = pd.DataFrame(data=cmatrix, index=labels, columns=labels)

fig, ax = plt.subplots(dpi=100)
sns.heatmap(df, linewidth=3, cmap='Greys', robust=True, annot=True, vmin=-100, vmax=1000, fmt="d", ax=ax, square=False, cbar=False)
plt.xlabel('True Class', c='green', size=14)
plt.ylabel('Predicted Class', c='red', size=14)
plt.show()

## Confusion Matrices for our models


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

Get the confusion matrix values for each of our classifiers in the previous section:


Use the variables for true and predicted labels that we set earlier: label_test, pred_NB, pred_KNN, pred_SVM


The solution for Naive Bayes has been provided. Create matrices for KNN and SVM.
</div>

### Naive Bayes Confusion Matrix

In [ ]:
# Naive Bayes c-matrix
nb_cm = confusion_matrix(label_test, pred_NB)
labels = ['setosa', 'versicolor', 'virginica']
df = pd.DataFrame(data=nb_cm, index=labels, columns=labels)

fig, ax = plt.subplots(dpi=100)
sns.heatmap(df, linewidth=3, cmap='Greys', robust=True, annot=True, vmin=-100, vmax=1000, fmt="d", ax=ax, square=False, cbar=False)
plt.xlabel('True Class', c='green', size=14)
plt.ylabel('Predicted Class', c='red', size=14)
plt.show()

### KNN Confusion Matrix

In [ ]:
# KNN c-matrix

### BEGIN SOLUTION
knn_cm = confusion_matrix(label_test, pred_KNN)
labels = ['setosa', 'versicolor', 'virginica']
df = pd.DataFrame(data=knn_cm, index=labels, columns=labels)

fig, ax = plt.subplots(dpi=100)
sns.heatmap(df, linewidth=3, cmap='Greys', robust=True, annot=True, vmin=-100, vmax=1000, fmt="d", ax=ax, square=False, cbar=False)
plt.xlabel('True Class', c='green', size=14)
plt.ylabel('Predicted Class', c='red', size=14)
plt.show()
### END SOLUTION

### SVM Confusion Matrix

In [ ]:
# SVM Bayes c-matrix

### BEGIN SOLUTION

svm_cm = confusion_matrix(label_test, pred_SVM)
labels = ['setosa', 'versicolor', 'virginica']
df = pd.DataFrame(data=svm_cm, index=labels, columns=labels)

fig, ax = plt.subplots(dpi=100)
sns.heatmap(df, linewidth=3, cmap='Greys', robust=True, annot=True, vmin=-100, vmax=1000, fmt="d", ax=ax, square=False, cbar=False)
plt.xlabel('True Class', c='green', size=14)
plt.ylabel('Predicted Class', c='red', size=14)
plt.show()

### END SOLUTION

## Model Metrics

In our PCA projection of the Iris dataset we saw some overlap between *I. virginica* and *I. versicolor*. We might expect our classifiers to struggle with these samples. 

In this section we will be assessing how our models performed on classifying samples with the true label **versicolor**.

In [ ]:
# First lets print the array representing the Naive Bayes confusion matrix:
print(nb_cm)

# We can extract the values we will need for versicolor like so
tp = nb_cm[1,1]
fn = nb_cm[2,1] + nb_cm[0,1]
fp = nb_cm[1,2] + nb_cm[1,0]
tn = nb_cm[0,0] + nb_cm[0,2] + nb_cm[2,0] + nb_cm[2,2] 

# Repeat this for the KNN and SVM confusion matrices
# Remember to use different variable names



## Accuracy

<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**How often is the classifier correct?**
    
Accuracy is the number of correct predictions made by the model over all predictions made. 

Accuracy is calculated as:<br><br>


$$
ACC = \frac{TP + TN}{TP + TN + FP + FN}
$$


<br>

Given the above equation, what is the **accuracy** of your classifiers? (Bayes, KNN, SVM)
    
</div>

In [ ]:
# Calculate and print Accuracy statistic using the formula above and pre-calculated tn, fp, fn, tp variables
#tn, fp, fn, tp 

### BEGIN SOLUTION
acc = (tp + tn) / (tp + tn + fp + fn)
### END SOLUTION

print(f'Accuracy: {acc}')

## Precision 

<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**When it predicts the positive result, how often is it correct?**

Precision is the proportion of predicted positives that truly are positives

$$
PPV = \frac{TP}{TP + FP}
$$

Given the above equation, what is the precision of your classifier?
</div>

In [ ]:
# Calculate and print Precision statistic using the formula above and pre-calculated tn, fp, fn, tp variables

### BEGIN SOLUTION
precision = tp / (tp+fp)
### END SOLUTION

print(f'Precision: {precision}')

## Recall (True Positive Rate)

<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**When it is actually the positive result, how often does it predict correctly?**
    
Recall is the proportion of actual positives that are correctly classified<br>

$$
TPR = \frac{TP}{TP + FN}
$$

Given the above equation, what is the recall of your classifier?
</div>

In [ ]:
# Calculate and print Recall statistic using the formula above and pre-calculated tn, fp, fn, tp variables

### BEGIN SOLUTION
recall = tp / (tp + fn)
### END SOLUTION

print(f'Recall: {recall}')

## False Positive Rate
<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**When it predicts a positive result, how often is it incorrect?**

$FPR = \frac{FP}{FP + TN}$

Given the above equation, what is the FPR of your classifier?
</div>

In [ ]:
# Calculate and print FPR using the formula above and pre-calculated tn, fp, fn, tp variables

### BEGIN SOLUTION
FPR = fp / (fp + tn)
### END SOLUTION

print(f'False positive rate: {FPR}')

## F1 score
<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**This is just the harmonic mean of precision and recall**
    
F1-score will penalise large discrepancies between precision and recall

$$
F_1 =  \frac{2*TP}{2*TP + FP + FN}
$$

Given the above equation, what is the F1 score of your classifier? 
</div>

In [ ]:
# Calculate and print F1 using the formula above and pre-calculated tn, fp, fn, tp variables

### BEGIN SOLUTION

f1 = 2 * tp / (2 * tp + fp + fn)

### END SOLUTION
print(f'F1 score: {f1}')

In [ ]:
# Hint at some helpful functions 
from sklearn.metrics import classification_report